In [94]:
import os

import yaml
from config import SETTINGS
from pathlib import Path
import pandas as pd

import seabeepy as sb

# Refactor `config.seabee.yaml` for niva-tidy missions

In [95]:
# Login to MinIO
minio_client = sb.storage.minio_login(
    user=SETTINGS.MINIO_ACCESS_ID, password=SETTINGS.MINIO_SECRET_KEY
)

In [96]:
# Parent directories containing flight folders to process
base_dirs = [
    Path(r"/home/notebook/shared-seabee-ns9879k/niva-tidy/2022"),
    Path(r"/home/notebook/shared-seabee-ns9879k/niva-tidy/2023"),
]


In [142]:

def get_spectrum_type(spec: str):
    if spec is None:
        return None
    if spec.lower() in ["rgb", "ms", "hsi"]:
        return spec.lower()
    
    if spec.lower() == "msi":
        return "ms"

def template_config() -> dict:
    return dict(
    grouping=None,
    area=None,
    datetime=None,
    spectrum_type=None,
    elevation=None,
    project=None,
    nfiles=None,
    organisation=None,
    creator_name=None,
    theme="habitat",
    mosaic=False,
    publish=True,
    #classify=True,
    )


def niva_name_to_config(name: str):

    parts = name.split("_")

    if len(parts) == 7:
        org, date, group, area, spect, elev, _ = parts
    elif len(parts) == 6:
        org, date, group, area, spect, elev = parts
    elif len(parts) == 5:
        org, date, group, area, spect = parts
        elev = None
    elif len(parts) == 4:
        org, date, group, area = parts
        spect = None
        elev = None

    print(f"org: {org}, date: {date}, group: {group}, area: {area}, spect: {spect}, elev: {elev}")
    return dict(
        grouping = group,
        area = area,
        datetime = date,
        spectrum_type = get_spectrum_type(spect),
        organisation = org,
        elevation = int(elev) if elev is not None and elev.isnumeric() else elev
    )


def parse_config(dir_path: Path):
    conf = {}
    config_path = dir_path / "config.seabee.yaml"
    if config_path.exists():
        with open(config_path, "r") as f:
            conf = yaml.safe_load(f)
    return conf


def write_new_config(path: Path, data):
    with open(path, "w") as yaml_file:
        yaml.dump(data, yaml_file, default_flow_style=False, allow_unicode=True)


def merge_conf(folder_conf: dict, existing_conf: dict) -> dict:
    """Merge the folder_conf with the existing_conf

    It is assumed that the folder_conf is the correct one for most fields
    """

    new_config = template_config()
    for key, val in folder_conf.items():
        new_config[key] = val
    for key in ["creator_name", "project", "mosaic", "publish", "theme"]:
        if key in existing_conf:
            if key == "creator_name" and "_" in existing_conf[key]:
                new_config[key] = " ".join(existing_conf[key].split("_"))
            else:
                new_config[key] = existing_conf[key]
    
    # when folder name is not too good try config file
    if new_config["spectrum_type"] is None:
        new_config["spectrum_type"] = get_spectrum_type(existing_conf["spectrum_type"])
    # some folder names have area with underscore try the config file instead
    # We can not allow underscore in the config file
    if "_" in existing_conf["area"]:
        new_config["area"] = existing_conf["area"].replace("_","-")
    new_config["grouping"] = f"{new_config['organisation'].lower()}-{existing_conf['project'].lower()}"
    return new_config

In [143]:
dir_list = [p for p in base_dirs for p in p.iterdir() if p.is_dir() and (p / "config.seabee.yaml").exists()]

In [144]:
tmp_dir = Path("./tmp")
tmp_dir.mkdir(exist_ok=True)

In [145]:
mission_dict = dict(folder_name= [], layer_name = [], valid = [])
mission_dict.update({k:[] for k in template_config().keys()})

for dir_path in dir_list:
    # Update config
    mission_name = os.path.split(dir_path)[-1]
    print(f"Checking {mission_name}")
    existing_conf = parse_config(dir_path)
    folder_conf = niva_name_to_config(mission_name)
    mission_dict["folder_name"].append(mission_name)
    try:
        new_conf = merge_conf(folder_conf, existing_conf)
        nfiles = len(sb.ortho.list_images(dir_path / "images", verbose=False))
        # validate all fields except nfiles
        new_conf["nfiles"] = nfiles if nfiles > 0 else 1
        sb.ortho.CONFIG_SCHEMA.validate(new_conf)
        write_new_config(tmp_dir / "config.seabee.yaml", new_conf)
        layer_name = sb.ortho.get_layer_name(tmp_dir)
    except Exception as e:
        print(f"Error in {mission_name}: {e}")
        new_conf["nfiles"] = nfiles
        mission_dict["layer_name"].append("error")
        mission_dict["valid"].append(False)
    else:
        new_conf["nfiles"] = nfiles
        mission_dict["layer_name"].append(layer_name)
        mission_dict["valid"].append(True)
        tmp_file = tmp_dir / f"{mission_name}.yaml"
        write_new_config(tmp_file, new_conf)
    
    for k, v in new_conf.items():
        mission_dict[k].append(v)
    

Checking niva_202208181236_kelpmap_vega-s_msi_60
org: niva, date: 202208181236, group: kelpmap, area: vega-s, spect: msi, elev: 60
Checking niva_202205180845_plastinoland_drammen_documentation_0
org: niva, date: 202205180845, group: plastinoland, area: drammen, spect: documentation, elev: 0
Checking spectrofly_202208180945_kelpmap_vega-s_msi_120
org: spectrofly, date: 202208180945, group: kelpmap, area: vega-s, spect: msi, elev: 120
Checking niva_202208191055_kelpmap_vega-n_msi_120
org: niva, date: 202208191055, group: kelpmap, area: vega-n, spect: msi, elev: 120
Checking spectrofly_202208311309_runde_remoy_msi_120
org: spectrofly, date: 202208311309, group: runde, area: remoy, spect: msi, elev: 120
Checking spectrofly_202209011343_runde_runde_rgb_120
org: spectrofly, date: 202209011343, group: runde, area: runde, spect: rgb, elev: 120
Checking spectrofly_202208181238_kelpmap_vega-s_msi_120
org: spectrofly, date: 202208181238, group: kelpmap, area: vega-s, spect: msi, elev: 120
Checkin

In [162]:
df = pd.DataFrame(mission_dict)

In [163]:
df[df.valid]

,folder_name,layer_name,valid,grouping,area,datetime,spectrum_type,elevation,project,nfiles,organisation,creator_name,theme,mosaic,publish
0,niva_202208181236_kelpmap_vega-s_msi_60,niva-kelpmap_vega-s_202208181236_ms_60m,True,niva-kelpmap,vega-s,202208181236,ms,60,kelpmap,4272,niva,Medyan Ghareeb,Habitat,False,True
1,niva_202205180845_plastinoland_drammen_documen...,niva-seabee_drammen_202205180845_ms,True,niva-seabee,drammen,202205180845,ms,0,SEABEE,112,niva,Medyan Ghareeb,Habitat,False,True
2,spectrofly_202208180945_kelpmap_vega-s_msi_120,spectrofly-kelpmap_vega-s_202208180945_ms_120m,True,spectrofly-kelpmap,vega-s,202208180945,ms,120,kelpmap,4850,spectrofly,Medyan Ghareeb,Habitat,False,True
3,niva_202208191055_kelpmap_vega-n_msi_120,niva-seabee_vega-n_202208191055_ms_120m,True,niva-seabee,vega-n,202208191055,ms,120,SEABEE,4956,niva,Medyan Ghareeb,Habitat,False,True
4,spectrofly_202208311309_runde_remoy_msi_120,spectrofly-runde_remoy_202208311309_ms_120m,True,spectrofly-runde,remoy,202208311309,ms,120,runde,0,spectrofly,Medyan Ghareeb,Habitat,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,niva_202308290947_halden_h29_rgb_80,niva-sabicas_halden-h29_202308290947_rgb_80m,True,niva-sabicas,halden-h29,202308290947,rgb,80,SABICAS,72,niva,Medyan Ghareeb,Habitat,False,True
184,niva_202310031115_io_husviksbaen_rgb_80,niva-seabee_husviksbaen_202310031115_rgb_80m,True,niva-seabee,husviksbaen,202310031115,rgb,80,SEABEE,196,niva,Medyan Ghareeb,Habitat,False,True
185,niva_202308301044_halden_h23_msi_80,niva-sabicas_halden-h23_202308301044_ms_80m,True,niva-sabicas,halden-h23,202308301044,ms,80,SABICAS,693,niva,Medyan Ghareeb,Habitat,False,True
186,niva_202309271230_ascomap_bringnes_msi_100,niva-ascomap_finnmark-bringnes_202309271230_ms...,True,niva-ascomap,finnmark-bringnes,202309271230,ms,100,ASCOMAP,4116,niva,Medyan Ghareeb,Habitat,False,True


In [150]:
df.groupby("area").first()


,folder_name,layer_name,valid,grouping,datetime,spectrum_type,elevation,project,nfiles,organisation,creator_name,theme,mosaic,publish
area,,,,,,,,,,,,,,
bekkensten,niva_202310021218_io_bekkensten_rgb_100,niva-seabee_bekkensten_202310021218_rgb_100m,True,niva-seabee,202310021218,rgb,100,SEABEE,200,niva,Medyan Ghareeb,Habitat,False,True
bonnebukta,niva_202310030959_io_bonnebukta_rgb_90,niva-seabee_bonnebukta_202310030959_rgb_90m,True,niva-seabee,202310030959,rgb,90,SEABEE,289,niva,Medyan Ghareeb,Habitat,False,True
drammen,niva_202205180845_plastinoland_drammen_documen...,niva-seabee_drammen_202205180845_ms,True,niva-seabee,202205180845,ms,0,SEABEE,112,niva,Medyan Ghareeb,Habitat,False,True
fagerstrand,niva_202310031253_io_fagerstrand_rgb_80,niva-seabee_fagerstrand_202310031253_rgb_80m,True,niva-seabee,202310031253,rgb,80,SEABEE,258,niva,Medyan Ghareeb,Habitat,False,True
finnmark-bringnes,niva_202309271147_ascomap_bringnes_msi_30,niva-ascomap_finnmark-bringnes_202309271147_ms...,True,niva-ascomap,202309271147,ms,30,ASCOMAP,2149,niva,Medyan Ghareeb,Habitat,False,True
finnmark-vassbukta,niva_202309281024_ascomap_vassbukta_msi_100,niva-ascomap_finnmark-vassbukta_202309281024_m...,True,niva-ascomap,202309281024,ms,100,ASCOMAP,3073,niva,Medyan Ghareeb,Habitat,False,True
halden-h20,niva_202308301136_halden_h20_msi_80,niva-sabicas_halden-h20_202308301136_ms_80m,True,niva-sabicas,202308301136,ms,80,SABICAS,301,niva,Medyan Ghareeb,Habitat,False,True
halden-h21,niva_202308301118_halden_h21_msi_80,niva-sabicas_halden-h21_202308301118_ms_80m,True,niva-sabicas,202308301118,ms,80,SABICAS,770,niva,Medyan Ghareeb,Habitat,False,True
halden-h22,niva_202308300752_halden_h22_msi_80,niva-sabicas_halden-h22_202308300752_ms_80m,True,niva-sabicas,202308300752,ms,80,SABICAS,378,niva,Medyan Ghareeb,Habitat,False,True


In [155]:
df.grouping.unique()

array(['niva-kelpmap', 'niva-seabee', 'spectrofly-kelpmap',
       'spectrofly-runde', 'ntnu-seabee', 'spectrofly-seabee',
       'niva-runderunderunderunde', 'niva-runde', 'spectrofly-srunde',
       'niva-ascomap', 'niva-sabicas'], dtype=object)

In [170]:
df.groupby("grouping").first()

,folder_name,layer_name,valid,area,datetime,spectrum_type,elevation,project,nfiles,organisation,creator_name,theme,mosaic,publish
grouping,,,,,,,,,,,,,,
niva-ascomap,niva_202309271147_ascomap_bringnes_msi_30,niva-ascomap_finnmark-bringnes_202309271147_ms...,True,finnmark-bringnes,202309271147,ms,30,ASCOMAP,2149,niva,Medyan Ghareeb,Habitat,False,True
niva-kelpmap,niva_202208181236_kelpmap_vega-s_msi_60,niva-kelpmap_vega-s_202208181236_ms_60m,True,vega-s,202208181236,ms,60,kelpmap,4272,niva,Medyan Ghareeb,Habitat,False,True
niva-runde,niva_202208310855_runde_remoy_msi_60,niva-runde_remoy_202208310855_ms_60m,True,remoy,202208310855,ms,60,runde,9258,niva,Medyan Ghareeb,Habitat,False,True
niva-runderunderunderunde,niva_202208311315_runde_remoy_msi_60,niva-runderunderunderunde_remoy_202208311315_m...,True,remoy,202208311315,ms,60,runderunderunderunde,4878,niva,Medyan Ghareeb,Habitat,False,True
niva-sabicas,niva_202308291402_halden_h24_msi_80,niva-sabicas_halden-h24_202308291402_ms_80m,True,halden-h24,202308291402,ms,80,SABICAS,896,niva,Medyan Ghareeb,Habitat,False,True
niva-seabee,niva_202205180845_plastinoland_drammen_documen...,niva-seabee_drammen_202205180845_ms,True,drammen,202205180845,ms,0,SEABEE,112,niva,Medyan Ghareeb,Habitat,False,True
ntnu-seabee,ntnu_202209011115_runde_runde_hsi_40,ntnu-seabee_runde_202209011115_hsi_40m,True,runde,202209011115,hsi,40,SEABEE,0,ntnu,Medyan Ghareeb,Habitat,False,True
spectrofly-kelpmap,spectrofly_202208180945_kelpmap_vega-s_msi_120,spectrofly-kelpmap_vega-s_202208180945_ms_120m,True,vega-s,202208180945,ms,120,kelpmap,4850,spectrofly,Medyan Ghareeb,Habitat,False,True
spectrofly-runde,spectrofly_202208311309_runde_remoy_msi_120,spectrofly-runde_remoy_202208311309_ms_120m,True,remoy,202208311309,ms,120,runde,0,spectrofly,Medyan Ghareeb,Habitat,False,True


In [171]:
df[df.grouping=='niva-kelpmap']

,folder_name,layer_name,valid,grouping,area,datetime,spectrum_type,elevation,project,nfiles,organisation,creator_name,theme,mosaic,publish
0,niva_202208181236_kelpmap_vega-s_msi_60,niva-kelpmap_vega-s_202208181236_ms_60m,True,niva-kelpmap,vega-s,202208181236,ms,60,kelpmap,4272,niva,Medyan Ghareeb,Habitat,False,True
9,niva_202208180814_kelpmap_vega-s_msi_115,niva-kelpmap_vega-s_202208180814_ms_115m,True,niva-kelpmap,vega-s,202208180814,ms,115,kelpmap,13710,niva,Medyan Ghareeb,Habitat,False,True
11,niva_202208181012_kelpmap_vega-s_rgb_60,niva-kelpmap_vega-s_202208181012_rgb_60m,True,niva-kelpmap,vega-s,202208181012,rgb,60,kelpmap,494,niva,Medyan Ghareeb,Habitat,False,True
26,niva_202208190944_kelpmap_vega-n_msi_120,niva-kelpmap_vega-n_202208190944_ms_120m,True,niva-kelpmap,vega-n,202208190944,ms,120,kelpmap,5034,niva,Medyan Ghareeb,Habitat,False,True
37,niva_202208181236_kelpmap_vega-s_rgb_60,niva-kelpmap_vega-s_202208181236_rgb_60m,True,niva-kelpmap,vega-s,202208181236,rgb,60,kelpmap,491,niva,Medyan Ghareeb,Habitat,False,True
40,niva_202208181012_kelpmap_vega-s_msi_60,niva-kelpmap_vega-s_202208181012_ms_60m,True,niva-kelpmap,vega-s,202208181012,ms,60,kelpmap,4308,niva,Medyan Ghareeb,Habitat,False,True
45,niva_202208180814_kelpmap_vega-s_rgb_115,niva-kelpmap_vega-s_202208180814_rgb_115m,True,niva-kelpmap,vega-s,202208180814,rgb,115,kelpmap,1804,niva,Medyan Ghareeb,Habitat,False,True
55,niva_202208191055_kelpmap_vega-n_rgb_120,niva-kelpmap_vega-n_202208191055_rgb_120m,True,niva-kelpmap,vega-n,202208191055,rgb,120,kelpmap,405,niva,Medyan Ghareeb,Habitat,False,True


In [173]:
areas = df.area.unique()
areas

array(['vega-s', 'drammen', 'vega-n', 'remoy', 'runde', 'landsteilene',
       'mfs', 'sverrehavn', 'vega', 'hellviktangen', 'knerten', 'sola',
       'site', 'juvika', 'larvik-olberg-all', 'larvik-olberg-s',
       'finnmark-bringnes', 'larvik-oldberg-all', 'halden-h24',
       'bonnebukta', 'olberg-all', 'olberg-s', 'ulvoya', 'halden-h36',
       'olberg-n', 'larvik-olberg-n', 'halden-h31', 'halden-h20',
       'halden-h25', 'halden-h35', 'halden-h37', 'halden-h22',
       'halden-h33', 'halden-h34', 'finnmark-vassbukta', 'halden-h29',
       'bekkensten', 'halden-h30', 'halden-h21', 'fagerstrand',
       'halden-h28', 'halden-h23', 'larvik-oldberg-s', 'husviksbaen'],
      dtype=object)

In [176]:
df.project.unique()

array(['kelpmap', 'SEABEE', 'runde', 'seabee', 'runderunderunderunde',
       'Srunde', 'ASCOMAP', 'SABICAS'], dtype=object)

In [175]:
df.groupby('project').first()

,folder_name,layer_name,valid,grouping,area,datetime,spectrum_type,elevation,nfiles,organisation,creator_name,theme,mosaic,publish
project,,,,,,,,,,,,,,
ASCOMAP,niva_202309271147_ascomap_bringnes_msi_30,niva-ascomap_finnmark-bringnes_202309271147_ms...,True,niva-ascomap,finnmark-bringnes,202309271147,ms,30,2149,niva,Medyan Ghareeb,Habitat,False,True
SABICAS,niva_202308291402_halden_h24_msi_80,niva-sabicas_halden-h24_202308291402_ms_80m,True,niva-sabicas,halden-h24,202308291402,ms,80,896,niva,Medyan Ghareeb,Habitat,False,True
SEABEE,niva_202205180845_plastinoland_drammen_documen...,niva-seabee_drammen_202205180845_ms,True,niva-seabee,drammen,202205180845,ms,0,112,niva,Medyan Ghareeb,Habitat,False,True
Srunde,spectrofly_202208311309_runde_remoy_rgb_120,spectrofly-srunde_remoy_202208311309_rgb_120m,True,spectrofly-srunde,remoy,202208311309,rgb,120,692,spectrofly,Medyan Ghareeb,Habitat,False,True
kelpmap,niva_202208181236_kelpmap_vega-s_msi_60,niva-kelpmap_vega-s_202208181236_ms_60m,True,niva-kelpmap,vega-s,202208181236,ms,60,4272,niva,Medyan Ghareeb,Habitat,False,True
runde,spectrofly_202208311309_runde_remoy_msi_120,spectrofly-runde_remoy_202208311309_ms_120m,True,spectrofly-runde,remoy,202208311309,ms,120,0,spectrofly,Medyan Ghareeb,Habitat,False,True
runderunderunderunde,niva_202208311315_runde_remoy_msi_60,niva-runderunderunderunde_remoy_202208311315_m...,True,niva-runderunderunderunde,remoy,202208311315,ms,60,4878,niva,Medyan Ghareeb,Habitat,False,True
seabee,niva_202203151133_seabee_landsteilene_rgb_80,niva-seabee_landsteilene_202203151133_rgb_80m,True,niva-seabee,landsteilene,202203151133,rgb,80,645,niva,Medyan Ghareeb,Habitat,False,True


In [116]:
df[df.area=="vega"]

,folder_name,layer_name,valid,grouping,area,datetime,spectrum_type,elevation,project,nfiles,organisation,creator_name,theme,mosaic,publish
19,niva_20220819_kelpmap_vega_documentation_0,kelpmap_vega_20220819_ms,True,kelpmap,vega,20220819,ms,0,SEABEE,14,niva,Medyan Ghareeb,Habitat,False,True
53,niva_20220818_kelpmap_vega_GT,kelpmap_vega_20220818_ms,True,kelpmap,vega,20220818,ms,None,SEABEE,0,niva,Medyan Ghareeb,Habitat,False,True


In [118]:
df[df.valid]

,folder_name,layer_name,valid,grouping,area,datetime,spectrum_type,elevation,project,nfiles,organisation,creator_name,theme,mosaic,publish
0,niva_202208181236_kelpmap_vega-s_msi_60,kelpmap_vega-s_202208181236_ms_60m,True,kelpmap,vega-s,202208181236,ms,60,kelpmap,4272,niva,Medyan Ghareeb,Habitat,False,True
1,niva_202205180845_plastinoland_drammen_documen...,plastinoland_drammen_202205180845_ms,True,plastinoland,drammen,202205180845,ms,0,SEABEE,112,niva,Medyan Ghareeb,Habitat,False,True
2,spectrofly_202208180945_kelpmap_vega-s_msi_120,kelpmap_vega-s_202208180945_ms_120m,True,kelpmap,vega-s,202208180945,ms,120,kelpmap,4850,spectrofly,Medyan Ghareeb,Habitat,False,True
3,niva_202208191055_kelpmap_vega-n_msi_120,kelpmap_vega-n_202208191055_ms_120m,True,kelpmap,vega-n,202208191055,ms,120,SEABEE,4956,niva,Medyan Ghareeb,Habitat,False,True
4,spectrofly_202208311309_runde_remoy_msi_120,runde_remoy_202208311309_ms_120m,True,runde,remoy,202208311309,ms,120,runde,0,spectrofly,Medyan Ghareeb,Habitat,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,niva_202308290947_halden_h29_rgb_80,halden_halden_h29_202308290947_rgb_80m,True,halden,halden_h29,202308290947,rgb,80,SABICAS,72,niva,Medyan Ghareeb,Habitat,False,True
184,niva_202310031115_io_husviksbaen_rgb_80,io_husviksbaen_202310031115_rgb_80m,True,io,husviksbaen,202310031115,rgb,80,SEABEE,196,niva,Medyan Ghareeb,Habitat,False,True
185,niva_202308301044_halden_h23_msi_80,halden_halden_h23_202308301044_ms_80m,True,halden,halden_h23,202308301044,ms,80,SABICAS,693,niva,Medyan Ghareeb,Habitat,False,True
186,niva_202309271230_ascomap_bringnes_msi_100,ascomap_finnmark_bringnes_202309271230_ms_100m,True,ascomap,finnmark_bringnes,202309271230,ms,100,ASCOMAP,4116,niva,Medyan Ghareeb,Habitat,False,True


# Take a look at

In [165]:
df[df.grouping=='niva-runderunderunderunde']

,folder_name,layer_name,valid,grouping,area,datetime,spectrum_type,elevation,project,nfiles,organisation,creator_name,theme,mosaic,publish
32,niva_202208311315_runde_remoy_msi_60,niva-runderunderunderunde_remoy_202208311315_m...,True,niva-runderunderunderunde,remoy,202208311315,ms,60,runderunderunderunde,4878,niva,Medyan Ghareeb,Habitat,False,True


In [166]:
df[df.grouping=='spectrofly-srunde']

,folder_name,layer_name,valid,grouping,area,datetime,spectrum_type,elevation,project,nfiles,organisation,creator_name,theme,mosaic,publish
57,spectrofly_202208311309_runde_remoy_rgb_120,spectrofly-srunde_remoy_202208311309_rgb_120m,True,spectrofly-srunde,remoy,202208311309,rgb,120,Srunde,692,spectrofly,Medyan Ghareeb,Habitat,False,True


In [164]:
df[~df.valid]

,folder_name,layer_name,valid,grouping,area,datetime,spectrum_type,elevation,project,nfiles,organisation,creator_name,theme,mosaic,publish
15,spectrofly_20220624_kelpmap_sverrehavn_uuv_usv,error,False,spectrofly-seabee,sverrehavn,20220624,ms,usv,SEABEE,0,spectrofly,Medyan Ghareeb,Habitat,False,True
29,spectrofly_20220820_kelpmap_sola_uuv_usv,error,False,spectrofly-seabee,sola,20220820,ms,usv,SEABEE,0,spectrofly,Medyan Ghareeb,Habitat,False,True
35,niva_202209_runde_runde_otter,error,False,niva-seabee,runde,202209,ms,None,SEABEE,0,niva,Medyan Ghareeb,Habitat,False,True
38,niva_yyyymmddhhmm_area_site_typ_alt,error,False,niva-seabee,site,yyyymmddhhmm,ms,alt,SEABEE,0,niva,Medyan Ghareeb,Habitat,False,True
41,niva_yyyymmddhhmm_area_site_typ_alt - Copy,error,False,niva-seabee,site,yyyymmddhhmm,ms,alt - Copy,SEABEE,0,niva,Medyan Ghareeb,Habitat,False,True
44,niva_202209_runde_runde_documentation_0,error,False,niva-seabee,runde,202209,ms,0,SEABEE,6,niva,Medyan Ghareeb,Habitat,False,True
132,niva_yyyymmddhhmm_area_site_typ_alt_compare,error,False,niva-seabee,site,yyyymmddhhmm,ms,alt,SEABEE,0,niva,Medyan Ghareeb,Habitat,False,True
142,niva_202310121354_oslo_fornebu_rgb_80,error,False,niva-seabee,larvik-olberg-s,202305051218,rgb,20,SEABEE,501,niva,Medyan Ghareeb,Habitat,False,True
152,spectorfly_20230911_seabee_stegenor_rgb_120_full,error,False,niva-sabicas,halden-h22,202308300741,rgb,80,SABICAS,64,niva,Medyan Ghareeb,Habitat,False,True
164,spectorfly_20230911_seabee_stegenor_rgb_120_fast,error,False,niva-seabee,husviksbaen,202310031129,ms,80,SEABEE,1596,niva,Medyan Ghareeb,Habitat,False,True
